In [1]:
import os
import time
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from pathlib import Path
import pennylane as qml
import gc
import subprocess
from datetime import datetime

# -------------------- GPU Selection --------------------
def get_free_gpu():
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
                            capture_output=True, text=True)
    memory = [int(x) for x in result.stdout.strip().split('\n')]
    print("GPU memory usage (MiB):", memory)
    best_gpu = np.argmin(memory)   # pick GPU with least used memory
    return best_gpu

GPU_ID = get_free_gpu()   # auto-select, or set manually
# GPU_ID = 5   # <-- uncomment to set manually
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
print(f"\nUsing GPU {GPU_ID}")

# -------------------- Device --------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Torch Device:", DEVICE)

# -------------------- Cleanup --------------------
torch.cuda.empty_cache()
gc.collect()

GPU memory usage (MiB): [8615, 219, 7399, 3, 3, 3, 3, 3]

Using GPU 3
Torch Device: cuda


20

In [2]:
# -------------------- Load Data --------------------
DRIVE_URL = "https://drive.google.com/uc?id=1PS0eB8dx8VMzVvxNUc6wBzsMRkEKJjWI"
df = pd.read_csv(DRIVE_URL)
print("Total rows:", len(df))
print("Total reactions:", df["Reaction"].nunique())

# -------------------- Physics Feature Engineering --------------------
M_p = 938.272088
M_n = 939.565420
epsilon = 1e-30
LN10 = np.log(10.0)

def get_nucleon_mass(Z, A):
    return Z * M_p + (A - Z) * M_n

mass1 = df.apply(lambda r: get_nucleon_mass(r["Z1"], r["A1"]), axis=1).values
mass2 = df.apply(lambda r: get_nucleon_mass(r["Z2"], r["A2"]), axis=1).values
mu_MeVc2 = (mass1 * mass2) / (mass1 + mass2 + 1e-12)
Ecm = df["E c.m."].astype(float).values
v_over_c = np.sqrt(np.clip(2 * Ecm / (mu_MeVc2 + epsilon), 0, np.inf))
e2_hbar_c = 1 / 137.035999
df["eta"] = (df["Z1"] * df["Z2"]) / (e2_hbar_c * (v_over_c + 1e-16))

log10_sigma_exp = np.log10(np.clip(df["σ"], 1e-30, np.inf))
log10_sigma_cal = np.log10(np.clip(df["σ cal"], 1e-30, np.inf))
log10_Ecm = np.log10(np.clip(df["E c.m."], 1e-30, np.inf))
log10_exp_term = (2 * np.pi * df["eta"]) / LN10

df["log10_S_exp"] = log10_sigma_exp + log10_Ecm + log10_exp_term
df["log10_S_cal"] = log10_sigma_cal + log10_Ecm + log10_exp_term
df["delta_log10_S"] = df["log10_S_exp"] - df["log10_S_cal"]

df["N1"] = df["A1"] - df["Z1"]
df["N2"] = df["A2"] - df["Z2"]
df["Z1Z2_over_Ecm"] = (df["Z1"] * df["Z2"]) / (df["E c.m."] + epsilon)

MAGIC = np.array([2, 8, 20, 28, 50, 82, 126])
def magic_dist(arr):
    return np.min(np.abs(arr[:, None] - MAGIC[None, :]), axis=1)

df["magic_dist_Z1"] = magic_dist(df["Z1"].values)
df["magic_dist_N1"] = magic_dist(df["N1"].values)
df["magic_dist_Z2"] = magic_dist(df["Z2"].values)
df["magic_dist_N2"] = magic_dist(df["N2"].values)

print("Feature engineering complete.")

Total rows: 3532
Total reactions: 213
Feature engineering complete.


In [3]:
# -------------------- 29 Features --------------------
features_train = [
    'E c.m.', 'Z1', 'N1', 'A1',
    'Z2', 'N2', 'A2', 'Q ( 2 n )',
    'Z1Z2_over_Ecm',
    'magic_dist_Z1','magic_dist_N1','magic_dist_Z2','magic_dist_N2',
    'Z3','N3','A3','β P','β T','R B','ħ ω',
    'Projectile_Mass_Actual', 'Target_Mass_Actual', 'Compound_Nucleus_Mass_Actual',
    'Compound_Nucleus_Sp','Compound_Nucleus_Sn',
    'Projectile_Binding_Energy','Target_Binding_Energy',
    'Compound_Nucleus_Binding_Energy','Compound_Nucleus_S2n'
]

# -------------------- Load Reaction Split --------------------
BASE_DIR = "mdn_70_10_20_optimized"
train_reacts = pd.read_csv(os.path.join(BASE_DIR, "train_reactions.csv"))["Reaction"].values
val_reacts   = pd.read_csv(os.path.join(BASE_DIR, "val_reactions.csv"))["Reaction"].values
test_reacts  = pd.read_csv(os.path.join(BASE_DIR, "test_reactions.csv"))["Reaction"].values

print("Train reactions:", len(train_reacts))
print("Val reactions:", len(val_reacts))
print("Test reactions:", len(test_reacts))

train_mask = df["Reaction"].isin(train_reacts)
val_mask   = df["Reaction"].isin(val_reacts)
test_mask  = df["Reaction"].isin(test_reacts)

# -------------------- Prepare Arrays --------------------
X_train_full = df.loc[train_mask | val_mask, features_train].values.astype(np.float32)
y_train_full = df.loc[train_mask | val_mask, "delta_log10_S"].values.astype(np.float32).reshape(-1,1)
X_test = df.loc[test_mask, features_train].values.astype(np.float32)
y_test = df.loc[test_mask, "delta_log10_S"].values.astype(np.float32).reshape(-1,1)

# -------------------- Standardize --------------------
scaler = StandardScaler().fit(X_train_full)
X_train_full_s = scaler.transform(X_train_full)
X_test_s = scaler.transform(X_test)

print("Training samples:", X_train_full_s.shape[0])
print("Test samples:", X_test_s.shape[0])

Train reactions: 149
Val reactions: 21
Test reactions: 43
Training samples: 2847
Test samples: 685


In [4]:
# ==================================================
# ADJUST THESE PARAMETERS FOR EACH EXPERIMENT
# ==================================================
n_qubits = 15
n_layers = 14
N_COMPONENTS = 5
BATCH_SIZE = 16
ACCUMULATION_STEPS = 2          # effective batch = 32
LAMBDA_ENTROPY = 0.05            # set to 0 to disable entropy
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
MAX_EPOCHS = 150
PATIENCE = 15
SEED = 42

# Effective batch size for logging
effective_batch = BATCH_SIZE * ACCUMULATION_STEPS
print(f"Qubits: {n_qubits}, Layers: {n_layers}, Components: {N_COMPONENTS}")
print(f"Physical batch: {BATCH_SIZE}, Accumulation steps: {ACCUMULATION_STEPS}")
print(f"Effective batch size: {effective_batch}")

# ---- Set seeds ----
torch.manual_seed(SEED)
np.random.seed(SEED)

# ---- Automatic naming ----
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"qmdn_q{n_qubits}_l{n_layers}_b{BATCH_SIZE}_acc{ACCUMULATION_STEPS}_s{SEED}_{timestamp}"
print("Model name:", model_name)

Qubits: 15, Layers: 14, Components: 5
Physical batch: 16, Accumulation steps: 2
Effective batch size: 32
Model name: qmdn_q15_l14_b16_acc2_s42_20260228_124052


In [5]:
dev = qml.device("default.qubit", wires=n_qubits, shots=None)
print("Using PennyLane device: default.qubit")

@qml.qnode(dev, interface="torch", diff_method="backprop")
def qnode(weights, inputs):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits), rotation="X")
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))   # shape (batch, 2**n_qubits)

Using PennyLane device: default.qubit


In [6]:
class QMDN(nn.Module):
    def __init__(self, in_dim, n_components=N_COMPONENTS, hidden_dim=32):
        super().__init__()
        self.n_components = n_components
        self.encoder = nn.Linear(in_dim, n_qubits)
        self.weight = nn.Parameter(0.01 * torch.randn(n_layers, n_qubits, 3))

        self.fc1 = nn.Linear(2**n_qubits, hidden_dim)
        self.fc_pi = nn.Linear(hidden_dim, n_components)
        self.fc_mu = nn.Linear(hidden_dim, n_components)
        self.fc_sigma = nn.Linear(hidden_dim, n_components)

    def forward(self, x):
        x = x.float()
        x_enc = torch.tanh(self.encoder(x))
        probs = qnode(self.weight, x_enc)
        probs = probs.to(x.dtype)          # cast to float32
        h = torch.relu(self.fc1(probs))
        pi_logits = self.fc_pi(h)
        mu = self.fc_mu(h)
        sigma_raw = self.fc_sigma(h)

        pi = F.softmax(pi_logits, dim=1)
        sigma = F.softplus(sigma_raw) + 1e-6
        return pi, mu, sigma

In [7]:
def mdn_loss(pi, mu, sigma, y):
    y = y.float()
    yexp = y.repeat(1, mu.shape[1])
    log_gauss = (
        -0.5 * ((yexp - mu) / sigma) ** 2
        - torch.log(sigma)
        - 0.5 * np.log(2 * np.pi)
    )
    log_mix = torch.logsumexp(torch.log(pi + 1e-12) + log_gauss, dim=1)
    return -log_mix.mean()

def mdn_loss_entropy(pi, mu, sigma, y, lambda_entropy):
    nll = mdn_loss(pi, mu, sigma, y)
    entropy = -torch.sum(pi * torch.log(pi + 1e-12), dim=1).mean()
    return nll - lambda_entropy * entropy

def make_loader(X, y, batch=BATCH_SIZE, shuffle=True):
    return DataLoader(
        TensorDataset(torch.tensor(X, dtype=torch.float32),
                      torch.tensor(y, dtype=torch.float32)),
        batch_size=batch,
        shuffle=shuffle,
        num_workers=0,
        pin_memory=True if DEVICE.type == 'cuda' else False
    )

# Train/Validation split
val_size = int(0.1 * len(X_train_full_s))
indices = np.random.permutation(len(X_train_full_s))
train_idx = indices[val_size:]
val_idx   = indices[:val_size]

train_loader = make_loader(X_train_full_s[train_idx], y_train_full[train_idx])
val_loader   = make_loader(X_train_full_s[val_idx],   y_train_full[val_idx], shuffle=False)
test_loader  = make_loader(X_test_s, y_test, shuffle=False)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))
print("Test batches:", len(test_loader))

Train batches: 161
Val batches: 18
Test batches: 43


In [8]:
model = QMDN(in_dim=X_train_full_s.shape[1]).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

best_val_nll = float("inf")
patience_counter = 0

# Tracking lists
epoch_pi_means = []      # average π per epoch
epoch_val_nll = []        # validation NLL per epoch
train_losses = []         # optional
val_losses = []           # optional

In [9]:
# =========================================================
# Training Loop with Per‑Epoch Component Monitoring (SAVED)
# =========================================================

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
best_val_nll = float("inf")
best_epoch = 0                     # <-- ADD THIS LINE
patience_counter = 0

# ---- Tracking lists ----
epoch_pi_means = []      # store average π per epoch
epoch_val_nll = []        # store validation NLL
train_losses = []         # optional
val_losses = []           # optional

print(f"\n🚀 Starting training: {model_name}\n")
for epoch in range(MAX_EPOCHS):
    model.train()
    train_loss = 0.0
    train_nll = 0.0
    optimizer.zero_grad()
    start_time = time.time()

    for batch_idx, (xb, yb) in enumerate(train_loader):
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)

        pi, mu, sigma = model(xb)

        if LAMBDA_ENTROPY > 0:
            loss = mdn_loss_entropy(pi, mu, sigma, yb, LAMBDA_ENTROPY) / ACCUMULATION_STEPS
        else:
            loss = mdn_loss(pi, mu, sigma, yb) / ACCUMULATION_STEPS

        nll = mdn_loss(pi, mu, sigma, yb) / ACCUMULATION_STEPS   # scaled for logging

        loss.backward()

        if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item() * xb.size(0) * ACCUMULATION_STEPS
        train_nll  += nll.item()  * xb.size(0) * ACCUMULATION_STEPS

    if (len(train_loader) % ACCUMULATION_STEPS) != 0:
        optimizer.step()
        optimizer.zero_grad()

    train_loss /= len(train_loader.dataset)
    train_nll  /= len(train_loader.dataset)
    train_losses.append(train_loss)

    # ---------- Validation ----------
    model.eval()
    val_loss = 0.0
    val_nll = 0.0
    pi_accum = []

    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pi, mu, sigma = model(xb)

            if LAMBDA_ENTROPY > 0:
                vloss = mdn_loss_entropy(pi, mu, sigma, yb, LAMBDA_ENTROPY)
            else:
                vloss = mdn_loss(pi, mu, sigma, yb)

            val_loss += vloss.item() * xb.size(0)
            val_nll  += mdn_loss(pi, mu, sigma, yb).item() * xb.size(0)
            pi_accum.append(pi.cpu())

    val_loss /= len(val_loader.dataset)
    val_nll  /= len(val_loader.dataset)
    val_losses.append(val_loss)

    # Average π across validation set
    pi_avg = torch.cat(pi_accum, dim=0).mean(dim=0).numpy()
    epoch_pi_means.append(pi_avg)
    epoch_val_nll.append(val_nll)

    scheduler.step(val_nll)
    epoch_time = time.time() - start_time
    current_lr = optimizer.param_groups[0]['lr']

    print(f"E{epoch:03d} | Time: {epoch_time:.2f}s | "
          f"train L={train_loss:.6f} NLL={train_nll:.6f} | "
          f"val L={val_loss:.6f} NLL={val_nll:.6f} | "
          f"π avg: {np.round(pi_avg,3)} | LR: {current_lr:.2e}")

    # ---------- Early Stopping ----------
    if val_nll < best_val_nll - 1e-4:
        best_val_nll = val_nll
        best_epoch = epoch          # <-- record the best epoch
        torch.save(model.state_dict(), f"{model_name}_best.pt")
        patience_counter = 0
        print(f"  *** New best model (epoch {epoch}, val NLL: {best_val_nll:.6f}) ***")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"🛑 Early stopping after epoch {epoch}")
            break

print("\n✅ Training finished. Best val NLL:", best_val_nll)


🚀 Starting training: qmdn_q15_l14_b16_acc2_s42_20260228_124052

E000 | Time: 204.74s | train L=0.434030 NLL=0.514159 | val L=0.341189 NLL=0.420613 | π avg: [0.237 0.158 0.208 0.148 0.248] | LR: 1.00e-03
  *** New best model (val NLL: 0.420613) ***
E001 | Time: 201.56s | train L=0.050841 NLL=0.126148 | val L=-0.071752 NLL=-0.003886 | π avg: [0.408 0.063 0.333 0.078 0.118] | LR: 1.00e-03
  *** New best model (val NLL: -0.003886) ***
E002 | Time: 207.79s | train L=-0.286693 NLL=-0.219086 | val L=-0.249795 NLL=-0.180798 | π avg: [0.41  0.068 0.315 0.1   0.106] | LR: 1.00e-03
  *** New best model (val NLL: -0.180798) ***
E003 | Time: 203.84s | train L=-0.422497 NLL=-0.352861 | val L=-0.374045 NLL=-0.305512 | π avg: [0.423 0.075 0.304 0.091 0.107] | LR: 1.00e-03
  *** New best model (val NLL: -0.305512) ***
E004 | Time: 201.12s | train L=-0.507093 NLL=-0.439688 | val L=-0.426090 NLL=-0.360039 | π avg: [0.449 0.071 0.304 0.077 0.099] | LR: 1.00e-03
  *** New best model (val NLL: -0.360039) *

In [10]:
# Save per-epoch data
np.save(f"{model_name}_epoch_pi_means.npy", np.array(epoch_pi_means))
np.save(f"{model_name}_epoch_val_nll.npy", np.array(epoch_val_nll))
np.save(f"{model_name}_train_losses.npy", np.array(train_losses))
np.save(f"{model_name}_val_losses.npy", np.array(val_losses))

# Save hyperparameters
with open(f"{model_name}_params.txt", "w") as f:
    f.write(f"n_qubits: {n_qubits}\n")
    f.write(f"n_layers: {n_layers}\n")
    f.write(f"N_COMPONENTS: {N_COMPONENTS}\n")
    f.write(f"BATCH_SIZE: {BATCH_SIZE}\n")
    f.write(f"ACCUMULATION_STEPS: {ACCUMULATION_STEPS}\n")
    f.write(f"effective_batch: {effective_batch}\n")
    f.write(f"LAMBDA_ENTROPY: {LAMBDA_ENTROPY}\n")
    f.write(f"LEARNING_RATE: {LEARNING_RATE}\n")
    f.write(f"WEIGHT_DECAY: {WEIGHT_DECAY}\n")
    f.write(f"MAX_EPOCHS: {MAX_EPOCHS}\n")
    f.write(f"PATIENCE: {PATIENCE}\n")
    f.write(f"SEED: {SEED}\n")
    f.write(f"best_val_nll: {best_val_nll:.6f}\n")
    f.write(f"best_epoch: {best_epoch+1}\n")
print("📁 Training history and parameters saved.")

NameError: name 'best_epoch' is not defined

In [11]:
# Load best model
model.load_state_dict(torch.load(f"{model_name}_best.pt", map_location=DEVICE))
model.eval()

def evaluate(loader, name):
    preds, truths = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            pi, mu, sigma = model(xb)
            delta_pred = torch.sum(pi * mu, dim=1, keepdim=True)
            preds.append(delta_pred.cpu().numpy())
            truths.append(yb.numpy())
    preds = np.vstack(preds)
    truths = np.vstack(truths)
    mse = mean_squared_error(truths, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(truths, preds)
    print(f"{name} -> MSE: {mse:.6f}, RMSE: {rmse:.6f}, R²: {r2:.6f}")
    return preds, truths

train_pred, train_true = evaluate(train_loader, "TRAIN")
val_pred, val_true     = evaluate(val_loader, "VALIDATION")
test_pred, test_true   = evaluate(test_loader, "TEST")

# Save predictions
np.savez(f"{model_name}_predictions.npz",
         train_pred=train_pred, train_true=train_true,
         val_pred=val_pred, val_true=val_true,
         test_pred=test_pred, test_true=test_true)
print("Predictions saved.")

/tmp/ipykernel_26279/3714618546.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{model_name}_best.pt", map_location=DEVICE))


TRAIN -> MSE: 0.062704, RMSE: 0.250409, R²: 0.203609
VALIDATION -> MSE: 0.101928, RMSE: 0.319262, R²: 0.090631
TEST -> MSE: 0.048081, RMSE: 0.219274, R²: -0.025806
Predictions saved.


In [12]:
def get_embeddings(loader):
    embeddings = []
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(DEVICE)
            x_enc = torch.tanh(model.encoder(xb))
            probs = qnode(model.weight, x_enc)   # shape (batch, 2**n_qubits)
            embeddings.append(probs.cpu().numpy())
    return np.vstack(embeddings)

train_emb = get_embeddings(train_loader)
val_emb   = get_embeddings(val_loader)
test_emb  = get_embeddings(test_loader)

np.savez(f"{model_name}_embeddings.npz",
         train_emb=train_emb, val_emb=val_emb, test_emb=test_emb)
print("Embeddings saved.")

Embeddings saved.


In [13]:
def get_mixture_params(loader):
    pi_list, mu_list, sigma_list = [], [], []
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(DEVICE)
            pi, mu, sigma = model(xb)
            pi_list.append(pi.cpu().numpy())
            mu_list.append(mu.cpu().numpy())
            sigma_list.append(sigma.cpu().numpy())
    return (np.vstack(pi_list), np.vstack(mu_list), np.vstack(sigma_list))

train_pi, train_mu, train_sigma = get_mixture_params(train_loader)
val_pi, val_mu, val_sigma       = get_mixture_params(val_loader)
test_pi, test_mu, test_sigma     = get_mixture_params(test_loader)

np.savez(f"{model_name}_mixtures.npz",
         train_pi=train_pi, train_mu=train_mu, train_sigma=train_sigma,
         val_pi=val_pi,   val_mu=val_mu,     val_sigma=val_sigma,
         test_pi=test_pi, test_mu=test_mu,   test_sigma=test_sigma)
print("Mixture parameters saved.")

Mixture parameters saved.


In [ ]:
# This cell is optional – run only if you plan to compute Quantum Fisher Information.
# It saves full statevectors for a few energy points per reaction.
if False:   # set to True to enable
    # Define a state‑returning QNode
    dev_state = qml.device("default.qubit", wires=n_qubits)
    @qml.qnode(dev_state, interface="torch")
    def state_qnode(weights, x):
        qml.templates.AngleEmbedding(x, wires=range(n_qubits), rotation="X")
        qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return qml.state()

    # Select representative samples (e.g., 5 per reaction)
    rep_indices = []
    for reaction in df["Reaction"].unique():
        rows = df[df["Reaction"] == reaction].sort_values("E c.m.")
        if len(rows) >= 3:
            idx = [rows.index[0], rows.index[len(rows)//2], rows.index[-1]]
        else:
            idx = rows.index.tolist()
        rep_indices.extend(idx)

    X_rep = X_scaled[rep_indices]
    reactions_rep = df.iloc[rep_indices]["Reaction"].values
    energy_rep = df.iloc[rep_indices]["E c.m."].values

    states = []
    with torch.no_grad():
        q_weights = model.weight.detach()
        for i in range(len(X_rep)):
            x_tensor = torch.tensor(X_rep[i], dtype=torch.float32).to(DEVICE)
            state = state_qnode(q_weights, x_tensor)
            states.append(state.cpu().numpy())
    states = np.array(states)

    np.savez(f"{model_name}_states.npz",
             states=states,
             indices=rep_indices,
             reactions=reactions_rep,
             energy=energy_rep)
    print("Statevectors saved.")